In [1]:
# -*- coding: utf-8 -*
# import necessary packages.
import pandas as pd
import numpy as np
import zipfile
import re
import datetime

# define a function so that we can get yesterday's date.

df_articles = pd.read_csv('../data/articles.csv', encoding = 'utf_8_sig')
df_android = pd.read_csv('../data/android_push2.csv', encoding = 'utf_8_sig')
df_ios = pd.read_csv('../data/ios_push2.csv', encoding = 'utf_8_sig')

In [2]:
df_ios.head()

,ios_title,Page_Views
0,大圍雙層九巴翻側 據悉1死10人受傷,3045
1,Juicy叮｜疑大聲講電話起爭執 兩女港鐵激戰扯頭髮互摑,2720
2,罕見意外｜北角女子疑被大量衣物壓著 倒斃房內,2625
3,確診者曾到新港中心H&M等4處及15間學校納入強制檢測,2544
4,迪士尼大抽獎送400張門票 得獎名單出爐,2537


# 1. data cleaning

In [3]:
# 2.1 Article name cleaning part.
def filter_tags(htmlstr):
    re_cdata=re.compile('//<!\[CDATA\[[^>]*//\]\]>',re.I)
    re_script=re.compile('<\s*script[^>]*>[^<]*<\s*/\s*script\s*>',re.I)
    re_style=re.compile('<\s*style[^>]*>[^<]*<\s*/\s*style\s*>',re.I)
    re_br=re.compile('<br\s*?/?>')
    re_h=re.compile('</?\w+[^>]*>')
    re_comment=re.compile('<!--[^>]*-->')
    s=re_cdata.sub('',htmlstr)
    s=re_script.sub('',s)
    s=re_style.sub('',s)
    s=re_br.sub('\n',s)
    s=re_h.sub('',s)
    s=re_comment.sub('',s)
    blank_line=re.compile('\n+')
    s=blank_line.sub('\n',s)
    s=replaceCharEntity(s)
    return s

def replaceCharEntity(htmlstr):
    CHAR_ENTITIES={'nbsp':' ','160':' ',
        'lt':'<','60':'<',
        'gt':'>','62':'>',
        'amp':'&','38':'&',
        'quot':'"','34':'"', '039':'\'', 
        'mdash':'-', 'ensp':' ',
        'hellip':'…','middot':'·', #'hellip':'…','middot':'·','hellip':'...','middot':'.'
        'deg':'°', 'ndash':'–',
        'bull':'•', '-':'—'}
   
    re_charEntity = re.compile(r'&#?(?P<name>\w+);')
    sz = re_charEntity.search(htmlstr)
    
    while sz:
        entity = sz.group()
        key = sz.group('name')
        try:
            htmlstr = re_charEntity.sub(CHAR_ENTITIES[key],htmlstr,1)
            sz = re_charEntity.search(htmlstr)
        except KeyError:
            htmlstr = re_charEntity.sub('',htmlstr,1)
            sz = re_charEntity.search(htmlstr)
    return (htmlstr)

def replace(s,re_exp,repl_string):
    return (re_exp.sub(repl_string,s))

In [4]:
for i in range(len(df_articles['title'])):
    if df_articles['title'][i] == 0:
        continue
    else:
        df_articles['title'][i] = filter_tags(df_articles['title'][i])
    
df_articles['title'] = df_articles['title'].str.strip()

<ipython-input-4-0892c18a6ac6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_articles['title'][i] = filter_tags(df_articles['title'][i])


In [5]:
for i in range(len(df_android['android_title'])):
    if df_android['android_title'][i] == 0:
        continue
    else:
        df_android['android_title'][i] = filter_tags(df_android['android_title'][i])
df_android['android_title'] = df_android['android_title'].str.strip()

<ipython-input-5-405afcc9be4d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_android['android_title'][i] = filter_tags(df_android['android_title'][i])


In [6]:
for i in range(len(df_ios['ios_title'])):
    if df_ios['ios_title'][i] == 0:
        continue
    else:
        df_ios['ios_title'][i] = filter_tags(df_ios['ios_title'][i])
df_ios['ios_title'] = df_ios['ios_title'].str.strip()

<ipython-input-6-ce14284cae0d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ios['ios_title'][i] = filter_tags(df_ios['ios_title'][i])


In [7]:
ch = []
pattern="[\u4e00-\u9fa5-0-9]+" 
regex = re.compile(pattern)
for i in range(len(df_articles)):
    result =  regex.findall(df_articles['title'][i])
    ch.append(result)
    
df_articles['key'] = ch

for i in range(len(df_articles)):
    df_articles['key'][i] = ''.join(df_articles['key'][i])

<ipython-input-7-3d98cb3c1701>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_articles['key'][i] = ''.join(df_articles['key'][i])


In [8]:
ch = []
pattern="[\u4e00-\u9fa5-0-9]+" 
regex = re.compile(pattern)
for i in range(len(df_android)):
    result =  regex.findall(df_android['android_title'][i])
    ch.append(result)
    
df_android['key'] = ch

for i in range(len(df_android)):
    df_android['key'][i] = ''.join(df_android['key'][i])

<ipython-input-8-287e8283fa6b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_android['key'][i] = ''.join(df_android['key'][i])


In [9]:
ch = []
pattern="[\u4e00-\u9fa5-0-9]+" 
regex = re.compile(pattern)
for i in range(len(df_ios)):
    result =  regex.findall(df_ios['ios_title'][i])
    ch.append(result)
    
df_ios['key'] = ch

for i in range(len(df_ios)):
    df_ios['key'][i] = ''.join(df_ios['key'][i])

<ipython-input-9-27f1dd659aba>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ios['key'][i] = ''.join(df_ios['key'][i])


In [11]:
df_articles

,id,title,complete_time,publish_time,key
0,41811.0,【最新消息】大圍雙層九巴翻側 據悉1死10人受傷,18/11/2021 23:50,18/11/2021 23:47,最新消息大圍雙層九巴翻側據悉1死10人受傷
1,41810.0,大圍雙層九巴翻側 10多人被困,18/11/2021 23:45,18/11/2021 23:42,大圍雙層九巴翻側10多人被困
2,41809.0,健康talk｜「黃金時段」入睡 減患心血管病風險,18/11/2021 23:00,18/11/2021 22:55,健康黃金時段入睡減患心血管病風險
3,41808.0,Foodpanda勞資雙方達成共識 罷工潮告一段落,18/11/2021 22:30,18/11/2021 22:27,勞資雙方達成共識罷工潮告一段落
4,41807.0,今日睇咗未?●料食肆最快本月25日全面用「安心出行」●本港氈酒名稱被英監管機構指似粗口冒犯民...,18/11/2021 21:50,18/11/2021 21:49,今日睇咗未料食肆最快本月25日全面用安心出行本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕...
...,...,...,...,...,...
125,41862.0,今日睇咗未?●冷鋒料凌晨橫過 日間徘徊19度●男女佐敦酒店窗邊激情 親熱片段網上瘋傳●炎明熹...,21/11/2021 21:55,21/11/2021 21:52,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳炎明熹肚瀉腳軟台上...
126,41861.0,美國返港23歲男初步確診變種病毒 愉景灣旭暉閣封區強檢,21/11/2021 19:40,21/11/2021 19:37,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢
127,41860.0,曼聯宣布解僱蘇斯克查 卡域克暫掌帥印,21/11/2021 19:25,21/11/2021 19:21,曼聯宣布解僱蘇斯克查卡域克暫掌帥印
128,41859.0,袁國勇:溝針試驗結果顯示沒問題 疫苗護照是公民責任,21/11/2021 17:45,21/11/2021 17:41,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任


# 2. create similarity matrix (articles as index, ios as column)

In [13]:
#Create a key-key binary matrix
key1 = df_articles.key.unique()
key2 = df_ios.key.unique()

keyMatrix = pd.DataFrame(columns=key2, index=key1)
keyMatrix.shape

(130, 925)

In [14]:
import numpy as np
import re
""" 
字符串余弦相似度
"""
def get_word_vector(s1, s2):
    """
    :param s1: 字符串1
    :param s2: 字符串2
    :return: 返回字符串切分后的向量
    """
    # 字符串中文按字分，英文按单词，数字按空格
    regEx = re.compile('[\\W]*')
    res = re.compile(r"([\u4e00-\u9fa5])")
    p1 = regEx.split(s1.lower())
    str1_list = []
    for str in p1:
        if res.split(str) == None:
            str1_list.append(str)
        else:
            ret = res.split(str)
            for ch in ret:
                str1_list.append(ch)
    # print(str1_list)
    p2 = regEx.split(s2.lower())
    str2_list = []
    for str in p2:
        if res.split(str) == None:
            str2_list.append(str)
        else:
            ret = res.split(str)
            for ch in ret:
                str2_list.append(ch)
    # print(str2_list)
    list_word1 = [w for w in str1_list if len(w.strip()) > 0]  # 去掉为空的字符
    list_word2 = [w for w in str2_list if len(w.strip()) > 0]  # 去掉为空的字符
    # 列出所有的词,取并集
    key_word = list(set(list_word1 + list_word2))
    # 给定形状和类型的用0填充的矩阵存储向量
    word_vector1 = np.zeros(len(key_word))
    word_vector2 = np.zeros(len(key_word))
    # 计算词频
    # 依次确定向量的每个位置的值
    for i in range(len(key_word)):
        # 遍历key_word中每个词在句子中的出现次数
        for j in range(len(list_word1)):
            if key_word[i] == list_word1[j]:
                word_vector1[i] += 1
        for k in range(len(list_word2)):
            if key_word[i] == list_word2[k]:
                word_vector2[i] += 1

    # 输出向量
    return word_vector1, word_vector2

def cos_dist(vec1, vec2):
    """
    :param vec1: 向量1
    :param vec2: 向量2
    :return: 返回两个向量的余弦相似度
    """
    dist1 = float(np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))
    return dist1

#if __name__ == '__main__':

In [15]:
for i in range(len(keyMatrix)):
    for j in range(len(keyMatrix.columns)):
        k1 = keyMatrix.index[i]
        k2 = keyMatrix.columns[j]
        v1, v2 = get_word_vector(k1, k2)
        cos_sim = cos_dist(v1,v2)
        keyMatrix.at[k1, k2] = cos_sim

In [69]:
# 获得 s1 行
df_index = []
df_columns = []
for i in range(len(keyMatrix)):
    s1 = keyMatrix.iloc[i,:]
    s1_argmax = s1[s1 == s1.max()].index[0]
    s1_score = s1[s1 == s1.max()][0]
    if  s1_score > 0.55:
        df_index.append(keyMatrix.index[i])
        df_columns.append(s1_argmax)

df_push = pd.DataFrame({'article_key': df_index,
                       'ios_key': df_columns})

df_push

,article_key,ios_key
0,最新消息大圍雙層九巴翻側據悉1死10人受傷,大圍雙層九巴翻側據悉1死10人受傷
1,大圍雙層九巴翻側10多人被困,大圍雙層九巴翻側10多人被困
2,健康黃金時段入睡減患心血管病風險,健康黃金時段入睡減患心血管病風險
3,勞資雙方達成共識罷工潮告一段落,勞資雙方達成共識罷工潮告一段落
4,今日睇咗未料食肆最快本月25日全面用安心出行本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕...,今日睇咗未本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕島5000萬隻紅蟹大遷徙奇觀
...,...,...
120,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳炎明熹肚瀉腳軟台上...,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳
121,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢
122,曼聯宣布解僱蘇斯克查卡域克暫掌帥印,英超官宣蘇斯克查下台卡域克暫掌帥印
123,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任


In [70]:
df_articles.rename(columns={'key':'article_key'},inplace=True)
df_ios.rename(columns={'key':'ios_key'},inplace=True)

In [71]:
df_push_sum1 = pd.merge(df_articles, df_push, on = 'article_key', how = 'left' )
df_push_sum2 = pd.merge(df_push_sum1, df_ios, on = 'ios_key', how = 'left' )
df_push_sum3 = df_push_sum2
df_push_sum3

,id,title,complete_time,publish_time,article_key,ios_key,ios_title,Page_Views
0,41811.0,【最新消息】大圍雙層九巴翻側 據悉1死10人受傷,18/11/2021 23:50,18/11/2021 23:47,最新消息大圍雙層九巴翻側據悉1死10人受傷,大圍雙層九巴翻側據悉1死10人受傷,大圍雙層九巴翻側 據悉1死10人受傷,3045.0
1,41810.0,大圍雙層九巴翻側 10多人被困,18/11/2021 23:45,18/11/2021 23:42,大圍雙層九巴翻側10多人被困,大圍雙層九巴翻側10多人被困,大圍雙層九巴翻側 10多人被困,1709.0
2,41809.0,健康talk｜「黃金時段」入睡 減患心血管病風險,18/11/2021 23:00,18/11/2021 22:55,健康黃金時段入睡減患心血管病風險,健康黃金時段入睡減患心血管病風險,健康talk｜「黃金時段」入睡 減患心血管病風險,2189.0
3,41808.0,Foodpanda勞資雙方達成共識 罷工潮告一段落,18/11/2021 22:30,18/11/2021 22:27,勞資雙方達成共識罷工潮告一段落,勞資雙方達成共識罷工潮告一段落,Foodpanda勞資雙方達成共識 罷工潮告一段落,1136.0
4,41807.0,今日睇咗未?●料食肆最快本月25日全面用「安心出行」●本港氈酒名稱被英監管機構指似粗口冒犯民...,18/11/2021 21:50,18/11/2021 21:49,今日睇咗未料食肆最快本月25日全面用安心出行本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕...,今日睇咗未本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕島5000萬隻紅蟹大遷徙奇觀,NaN,NaN
...,...,...,...,...,...,...,...,...
126,41862.0,今日睇咗未?●冷鋒料凌晨橫過 日間徘徊19度●男女佐敦酒店窗邊激情 親熱片段網上瘋傳●炎明熹...,21/11/2021 21:55,21/11/2021 21:52,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳炎明熹肚瀉腳軟台上...,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳,NaN,NaN
127,41861.0,美國返港23歲男初步確診變種病毒 愉景灣旭暉閣封區強檢,21/11/2021 19:40,21/11/2021 19:37,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢,美國返港23歲男初步確診變種病毒 愉景灣旭暉閣封區強檢,900.0
128,41860.0,曼聯宣布解僱蘇斯克查 卡域克暫掌帥印,21/11/2021 19:25,21/11/2021 19:21,曼聯宣布解僱蘇斯克查卡域克暫掌帥印,英超官宣蘇斯克查下台卡域克暫掌帥印,英超｜官宣！ 蘇斯克查下台 卡域克暫掌帥印,33.0
129,41859.0,袁國勇:溝針試驗結果顯示沒問題 疫苗護照是公民責任,21/11/2021 17:45,21/11/2021 17:41,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任,袁國勇:溝針試驗結果顯示沒問題 疫苗護照是公民責任,1208.0


In [72]:
df_push_sum3.rename(columns={'Page_Views':'Page_Views_ios'},inplace=True)

In [73]:
df_push_sum3.drop(['ios_key', 'publish_time', 'ios_title'], axis=1, inplace = True)
df_push_sum3 = df_push_sum3[['id', 'title', 'complete_time', 'article_key', 'Page_Views_ios']]
df_push_sum3

,id,title,complete_time,article_key,Page_Views_ios
0,41811.0,【最新消息】大圍雙層九巴翻側 據悉1死10人受傷,18/11/2021 23:50,最新消息大圍雙層九巴翻側據悉1死10人受傷,3045.0
1,41810.0,大圍雙層九巴翻側 10多人被困,18/11/2021 23:45,大圍雙層九巴翻側10多人被困,1709.0
2,41809.0,健康talk｜「黃金時段」入睡 減患心血管病風險,18/11/2021 23:00,健康黃金時段入睡減患心血管病風險,2189.0
3,41808.0,Foodpanda勞資雙方達成共識 罷工潮告一段落,18/11/2021 22:30,勞資雙方達成共識罷工潮告一段落,1136.0
4,41807.0,今日睇咗未?●料食肆最快本月25日全面用「安心出行」●本港氈酒名稱被英監管機構指似粗口冒犯民...,18/11/2021 21:50,今日睇咗未料食肆最快本月25日全面用安心出行本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕...,NaN
...,...,...,...,...,...
126,41862.0,今日睇咗未?●冷鋒料凌晨橫過 日間徘徊19度●男女佐敦酒店窗邊激情 親熱片段網上瘋傳●炎明熹...,21/11/2021 21:55,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳炎明熹肚瀉腳軟台上...,NaN
127,41861.0,美國返港23歲男初步確診變種病毒 愉景灣旭暉閣封區強檢,21/11/2021 19:40,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢,900.0
128,41860.0,曼聯宣布解僱蘇斯克查 卡域克暫掌帥印,21/11/2021 19:25,曼聯宣布解僱蘇斯克查卡域克暫掌帥印,33.0
129,41859.0,袁國勇:溝針試驗結果顯示沒問題 疫苗護照是公民責任,21/11/2021 17:45,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任,1208.0


# 3. create another matrix (articles&ios as index, android as column)

In [74]:
#Create a key-key binary matrix
key3 = df_push_sum3.article_key.unique()
key4 = df_android.key.unique()

keyMatrix2 = pd.DataFrame(columns=key4, index=key3)
keyMatrix2.shape

AttributeError: 'DataFrame' object has no attribute 'key'

In [75]:
for i in range(len(keyMatrix2)):
    for j in range(len(keyMatrix2.columns)):
        k3 = keyMatrix2.index[i]
        k4 = keyMatrix2.columns[j]
        v1, v2 = get_word_vector(k3, k4)
        cos_sim = cos_dist(v1,v2)
        keyMatrix2.at[k3, k4] = cos_sim

<ipython-input-14-2b0f369f8abe>:62: RuntimeWarning: invalid value encountered in double_scalars
  dist1 = float(np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))


In [76]:
# 获得 s1 行
df_index2 = []
df_columns2 = []
for i in range(len(keyMatrix2)):
    s1 = keyMatrix2.iloc[i,:]
    s1_argmax = s1[s1 == s1.max()].index[0]
    s1_score = s1[s1 == s1.max()][0]
    if s1_score > 0.55:
        df_index2.append(keyMatrix2.index[i])
        df_columns2.append(s1_argmax)

df_push2 = pd.DataFrame({'article_key': df_index2,
                       'android_key': df_columns2})

df_push2

,article_key,android_key
0,最新消息大圍雙層九巴翻側據悉1死10人受傷,大圍雙層九巴翻側據悉1死10人受傷
1,大圍雙層九巴翻側10多人被困,大圍雙層九巴翻側10多人被困
2,健康黃金時段入睡減患心血管病風險,健康黃金時段入睡減患心血管病風險
3,勞資雙方達成共識罷工潮告一段落,勞資雙方達成共識罷工潮告一段落
4,今日睇咗未料食肆最快本月25日全面用安心出行本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕...,今日睇咗未本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕島5000萬隻紅蟹大遷徙奇觀
...,...,...
120,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳炎明熹肚瀉腳軟台上...,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳
121,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢
122,曼聯宣布解僱蘇斯克查卡域克暫掌帥印,英超官宣蘇斯克查下台卡域克暫掌帥印
123,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任


In [51]:
df_android.rename(columns={'key':'android_key', 'Page_Views':'Page_Views_android'},inplace=True)

In [77]:
df_push_sum4 = pd.merge(df_push_sum3, df_push2, on = 'article_key', how = 'left' )
df_push_sum4

,id,title,complete_time,article_key,Page_Views_ios,android_key
0,41811.0,【最新消息】大圍雙層九巴翻側 據悉1死10人受傷,18/11/2021 23:50,最新消息大圍雙層九巴翻側據悉1死10人受傷,3045.0,大圍雙層九巴翻側據悉1死10人受傷
1,41810.0,大圍雙層九巴翻側 10多人被困,18/11/2021 23:45,大圍雙層九巴翻側10多人被困,1709.0,大圍雙層九巴翻側10多人被困
2,41809.0,健康talk｜「黃金時段」入睡 減患心血管病風險,18/11/2021 23:00,健康黃金時段入睡減患心血管病風險,2189.0,健康黃金時段入睡減患心血管病風險
3,41808.0,Foodpanda勞資雙方達成共識 罷工潮告一段落,18/11/2021 22:30,勞資雙方達成共識罷工潮告一段落,1136.0,勞資雙方達成共識罷工潮告一段落
4,41807.0,今日睇咗未?●料食肆最快本月25日全面用「安心出行」●本港氈酒名稱被英監管機構指似粗口冒犯民...,18/11/2021 21:50,今日睇咗未料食肆最快本月25日全面用安心出行本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕...,NaN,今日睇咗未本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕島5000萬隻紅蟹大遷徙奇觀
...,...,...,...,...,...,...
126,41862.0,今日睇咗未?●冷鋒料凌晨橫過 日間徘徊19度●男女佐敦酒店窗邊激情 親熱片段網上瘋傳●炎明熹...,21/11/2021 21:55,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳炎明熹肚瀉腳軟台上...,NaN,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳
127,41861.0,美國返港23歲男初步確診變種病毒 愉景灣旭暉閣封區強檢,21/11/2021 19:40,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢,900.0,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢
128,41860.0,曼聯宣布解僱蘇斯克查 卡域克暫掌帥印,21/11/2021 19:25,曼聯宣布解僱蘇斯克查卡域克暫掌帥印,33.0,英超官宣蘇斯克查下台卡域克暫掌帥印
129,41859.0,袁國勇:溝針試驗結果顯示沒問題 疫苗護照是公民責任,21/11/2021 17:45,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任,1208.0,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任


In [81]:
df_push_sum5 = pd.merge(df_push_sum4, df_android, on = 'android_key', how = 'left')
df_push_sum5

,id,title,complete_time,article_key,Page_Views_ios,android_key,android_title,Page_Views_android
0,41811.0,【最新消息】大圍雙層九巴翻側 據悉1死10人受傷,18/11/2021 23:50,最新消息大圍雙層九巴翻側據悉1死10人受傷,3045.0,大圍雙層九巴翻側據悉1死10人受傷,大圍雙層九巴翻側 據悉1死10人受傷,2030.0
1,41810.0,大圍雙層九巴翻側 10多人被困,18/11/2021 23:45,大圍雙層九巴翻側10多人被困,1709.0,大圍雙層九巴翻側10多人被困,大圍雙層九巴翻側 10多人被困,575.0
2,41809.0,健康talk｜「黃金時段」入睡 減患心血管病風險,18/11/2021 23:00,健康黃金時段入睡減患心血管病風險,2189.0,健康黃金時段入睡減患心血管病風險,健康talk｜「黃金時段」入睡 減患心血管病風險,2595.0
3,41808.0,Foodpanda勞資雙方達成共識 罷工潮告一段落,18/11/2021 22:30,勞資雙方達成共識罷工潮告一段落,1136.0,勞資雙方達成共識罷工潮告一段落,Foodpanda勞資雙方達成共識 罷工潮告一段落,1315.0
4,41807.0,今日睇咗未?●料食肆最快本月25日全面用「安心出行」●本港氈酒名稱被英監管機構指似粗口冒犯民...,18/11/2021 21:50,今日睇咗未料食肆最快本月25日全面用安心出行本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕...,NaN,今日睇咗未本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕島5000萬隻紅蟹大遷徙奇觀,今日睇咗未? 本港氈酒名稱被英監管機構指似粗口冒犯民眾／澳洲聖誕島5000萬隻紅蟹大遷徙奇觀,2896.0
...,...,...,...,...,...,...,...,...
128,41862.0,今日睇咗未?●冷鋒料凌晨橫過 日間徘徊19度●男女佐敦酒店窗邊激情 親熱片段網上瘋傳●炎明熹...,21/11/2021 21:55,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳炎明熹肚瀉腳軟台上...,NaN,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳,今日睇咗未？冷鋒料凌晨橫過 日間徘徊19度／男女佐敦酒店窗邊激情親熱片段網上瘋傳,3068.0
129,41861.0,美國返港23歲男初步確診變種病毒 愉景灣旭暉閣封區強檢,21/11/2021 19:40,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢,900.0,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢,美國返港23歲男初步確診變種病毒 愉景灣旭暉閣封區強檢,1383.0
130,41860.0,曼聯宣布解僱蘇斯克查 卡域克暫掌帥印,21/11/2021 19:25,曼聯宣布解僱蘇斯克查卡域克暫掌帥印,33.0,英超官宣蘇斯克查下台卡域克暫掌帥印,英超｜官宣！ 蘇斯克查下台 卡域克暫掌帥印,164.0
131,41859.0,袁國勇:溝針試驗結果顯示沒問題 疫苗護照是公民責任,21/11/2021 17:45,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任,1208.0,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任,袁國勇:溝針試驗結果顯示沒問題 疫苗護照是公民責任,2344.0


In [82]:
df_push_sum6 = df_push_sum5[['id', 'title', 'complete_time', 'article_key', 'Page_Views_ios', 'Page_Views_android']]

In [83]:
df_push_sum7  = df_push_sum6

In [84]:
df_push_sum7

,id,title,complete_time,article_key,Page_Views_ios,Page_Views_android
0,41811.0,【最新消息】大圍雙層九巴翻側 據悉1死10人受傷,18/11/2021 23:50,最新消息大圍雙層九巴翻側據悉1死10人受傷,3045.0,2030.0
1,41810.0,大圍雙層九巴翻側 10多人被困,18/11/2021 23:45,大圍雙層九巴翻側10多人被困,1709.0,575.0
2,41809.0,健康talk｜「黃金時段」入睡 減患心血管病風險,18/11/2021 23:00,健康黃金時段入睡減患心血管病風險,2189.0,2595.0
3,41808.0,Foodpanda勞資雙方達成共識 罷工潮告一段落,18/11/2021 22:30,勞資雙方達成共識罷工潮告一段落,1136.0,1315.0
4,41807.0,今日睇咗未?●料食肆最快本月25日全面用「安心出行」●本港氈酒名稱被英監管機構指似粗口冒犯民...,18/11/2021 21:50,今日睇咗未料食肆最快本月25日全面用安心出行本港氈酒名稱被英監管機構指似粗口冒犯民眾澳洲聖誕...,NaN,2896.0
...,...,...,...,...,...,...
128,41862.0,今日睇咗未?●冷鋒料凌晨橫過 日間徘徊19度●男女佐敦酒店窗邊激情 親熱片段網上瘋傳●炎明熹...,21/11/2021 21:55,今日睇咗未冷鋒料凌晨橫過日間徘徊19度男女佐敦酒店窗邊激情親熱片段網上瘋傳炎明熹肚瀉腳軟台上...,NaN,3068.0
129,41861.0,美國返港23歲男初步確診變種病毒 愉景灣旭暉閣封區強檢,21/11/2021 19:40,美國返港23歲男初步確診變種病毒愉景灣旭暉閣封區強檢,900.0,1383.0
130,41860.0,曼聯宣布解僱蘇斯克查 卡域克暫掌帥印,21/11/2021 19:25,曼聯宣布解僱蘇斯克查卡域克暫掌帥印,33.0,164.0
131,41859.0,袁國勇:溝針試驗結果顯示沒問題 疫苗護照是公民責任,21/11/2021 17:45,袁國勇溝針試驗結果顯示沒問題疫苗護照是公民責任,1208.0,2344.0


In [85]:
Page_Views_sum = []
for i in range(len(df_push_sum7)):
    pv_sum = df_push_sum7['Page_Views_android'][i]+df_push_sum7['Page_Views_ios'][i]
    Page_Views_sum.append(pv_sum)
    
df_push_sum7['Page_Views_Sum'] = Page_Views_sum

<ipython-input-85-3421adc001db>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_push_sum7['Page_Views_Sum'] = Page_Views_sum


In [86]:
df_push_sum8 = df_push_sum7[['id', 'title', 'complete_time', 'Page_Views_ios', 'Page_Views_android', 'Page_Views_Sum']]

In [89]:
df_push_sum8.to_csv('../data/sum_v2.csv', encoding = 'utf_8_sig')

In [88]:
df_push_sum8

,id,title,complete_time,Page_Views_ios,Page_Views_android,Page_Views_Sum
0,41811.0,【最新消息】大圍雙層九巴翻側 據悉1死10人受傷,18/11/2021 23:50,3045.0,2030.0,5075.0
1,41810.0,大圍雙層九巴翻側 10多人被困,18/11/2021 23:45,1709.0,575.0,2284.0
2,41809.0,健康talk｜「黃金時段」入睡 減患心血管病風險,18/11/2021 23:00,2189.0,2595.0,4784.0
3,41808.0,Foodpanda勞資雙方達成共識 罷工潮告一段落,18/11/2021 22:30,1136.0,1315.0,2451.0
4,41807.0,今日睇咗未?●料食肆最快本月25日全面用「安心出行」●本港氈酒名稱被英監管機構指似粗口冒犯民...,18/11/2021 21:50,NaN,2896.0,NaN
...,...,...,...,...,...,...
128,41862.0,今日睇咗未?●冷鋒料凌晨橫過 日間徘徊19度●男女佐敦酒店窗邊激情 親熱片段網上瘋傳●炎明熹...,21/11/2021 21:55,NaN,3068.0,NaN
129,41861.0,美國返港23歲男初步確診變種病毒 愉景灣旭暉閣封區強檢,21/11/2021 19:40,900.0,1383.0,2283.0
130,41860.0,曼聯宣布解僱蘇斯克查 卡域克暫掌帥印,21/11/2021 19:25,33.0,164.0,197.0
131,41859.0,袁國勇:溝針試驗結果顯示沒問題 疫苗護照是公民責任,21/11/2021 17:45,1208.0,2344.0,3552.0
